# Convert Calvados to Faunus-RS format

This converts CSV files to JSON files for Faunus.

## Further processing to YAML

The JSON file should be converted to YAML and here it's important to convert the `hydrophobicity` column which is currently a `string`. It should instead represent a custom datatype, `Lambda` or `λ` as signified with `!`.
In short, convert to YAML and remove the quotes, `"`.

## References

- [Calvados2](https://open-research-europe.ec.europa.eu/articles/2-94/v2)
- [Calvados3](https://onlinelibrary.wiley.com/doi/10.1002/pro.5172)
- [Github](https://github.com/KULL-Centre/CALVADOS)

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os.path

In [ ]:
def convert_calvados3():
    csvfile = "calvados3.csv"
    if not os.path.isfile(csvfile):
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/KULL-Centre/CALVADOS/refs/heads/main/examples/single_MDP/input/residues_CALVADOS3.csv",
            csvfile,
        )
    df = pd.read_csv(
        csvfile, usecols=["three", "MW", "sigmas", "sigmas", "q", "lambdas"]
    )
    df.columns = ["name", "mass", "hydrophobicity", "sigma", "charge"]
    df["hydrophobicity"] = df["hydrophobicity"].map("!Lambda {:.5}".format)
    df["ε"] = 0.8368  # kJ/mol
    df["sigma"] *= 10  # nm -> angstrom
    df.to_json("calvados3.json", orient="records", force_ascii=False)
    return df

In [ ]:
df = convert_calvados3()
df

# Plot potential

In [ ]:
σ = 6.0  # angstrom
ε = 0.8368  # kJ/mol
λ = 0.5
rc = 3 * σ  # angstrom


def lj(r):
    return 4 * ε * ((σ / r) ** 12 - (σ / r) ** 6)


def ah(r):
    if r > rc:
        return 0.0
    if r <= σ * 2 ** (1 / 6):
        return lj(r) - λ * lj(rc) + ε * (1 - λ)
    return λ * (lj(r) - lj(rc))


r = np.linspace(0.9 * σ, rc, 50)
f = np.vectorize(ah)

plt.plot(r / σ, lj(r) / ε, label="Lennard-Jones")
plt.plot(r / σ, f(r) / ε, label="Ashbaugh-Hatch")

plt.xlabel("r/σ")
plt.ylabel("u(r)/ε")
plt.legend(loc=0, frameon=False)